In [8]:
import requests
import json
import pandas as pd
import os
import sys
import urllib.parse
import nltk
import string
import copy
import re
import numpy as np
from collections import defaultdict

In [50]:
#1. Consider only the alphabetic part of the LCC code
def lcc_to_scimago_area_category(lcc_code):
    alphabetic_code = re.findall('[a-zA-Z]+', lcc_code)
    if len(alphabetic_code) == 0:
        print("Error: invalid LCC code "+lcc_code+": doesn't start with alphabetic characters")

    alphabetic_code = alphabetic_code[0].upper()
    query_df = lcc_lookup_df.loc[lcc_lookup_df['lcc_code'] == alphabetic_code]
    lcc_subject = None
    if len(query_df) > 0:
        lcc_subject = query_df["lcc_subject"].values[0]
    else:
        return ("", "")
        #print("Error: invalid LCC code "+lcc_code+": not found")

    area = ""
    category = ""
    #2. Checks whether the value of the LCC subject is also a Scimago subject area
    query_df = scimago_lookup_df.loc[scimago_lookup_df['area'].str.lower() == lcc_subject.lower()]
    if len(query_df) > 0:
        area = query_df["area"].values[0]
        category = area + " (miscellaneous)"

    #3. Checks whether the value of the LCC subject is also a Scimago subject category
    # some category names might be included in more than one area -> We take into consideration all the alternativies
    else:
        query_df = scimago_lookup_df.loc[scimago_lookup_df['category'].str.lower() == lcc_subject.lower()]
        if len(query_df) > 0:
            area = ""
            category = ""
            for a_index in range(0,len(query_df["area"].values)):
                area = area + query_df["area"].values[a_index] +";;"
                category = category + query_df["category"].values[a_index] +";;"
            area = area[:-2]
            category = category[:-2]

    return (area, category)

In [51]:
lcc_lookup_df = pd.read_csv("script/data/lcc_lookup.csv")
scimago_lookup_df = pd.read_csv("script/data/scimago_lookup.csv")
df_all_cits_inp1 = pd.read_csv('annotation/all_cits(normalized).csv')

l_rows = []
#cited_magid	cited_doi	cited_ref_hints(author)	cited_retraction_year	cited_publication_year	citing_doi	citing_title	citing_author	citing_venue	venue_id	venue_type	venue_area	venue_category	citing_publisher	citing_publication_year	source	period	fulltext	retracted	notes
for k, row in df_all_cits_inp1.iterrows():
    venue_id = str(row["venue_id"]).lower()
    if venue_id.startswith("isbn"):
        if "lcc" in venue_id:
            lcc_code = venue_id.split("lcc:")[1].strip()
            scimago_res = lcc_to_scimago_area_category(lcc_code)
            row["venue_area"] = scimago_res[0]
            row["venue_category"] = scimago_res[1]
    l_rows.append(row)

In [52]:
pd.DataFrame(l_rows).to_csv('annotation/all_cits(lcc_to_scimago).csv',index=False)

In [55]:
df_all_cits = pd.read_csv('annotation/all_cits.csv')

In [64]:
area_dict = defaultdict(int)
for k, row in df_all_cits.iterrows():
    a = str(row["venue_area"])
    for a_part in a.split(";;"):
        a_part = a_part.lower().strip()
        area_dict[a_part] += 1

In [67]:
area_dict

defaultdict(int,
            {'arts and humanities': 161,
             'medicine': 259,
             'social sciences': 183,
             'psychology': 167,
             'nan': 113,
             'mathematics': 15,
             'engineering': 37,
             'nursing': 26,
             'biochemistry, genetics and molecular biology': 16,
             'computer science': 23,
             'business, management and accounting': 55,
             'multidisciplinary': 11,
             'chemical engineering': 5,
             'chemistry': 3,
             'energy': 14,
             'neuroscience': 24,
             'earth and planetary sciences': 36,
             'environmental science': 25,
             'economics, econometrics and finance': 22,
             '': 18,
             'geography, planning and development': 2,
             'pharmacology, toxicology and pharmaceutics': 6,
             'materials science': 8,
             'decision sciences': 4,
             'agricultural and biological 

In [68]:
for k, val in area_dict.items():
    print(k,val)

arts and humanities 161
medicine 259
social sciences 183
psychology 167
nan 113
mathematics 15
engineering 37
nursing 26
biochemistry, genetics and molecular biology 16
computer science 23
business, management and accounting 55
multidisciplinary 11
chemical engineering 5
chemistry 3
energy 14
neuroscience 24
earth and planetary sciences 36
environmental science 25
economics, econometrics and finance 22
 18
geography, planning and development 2
pharmacology, toxicology and pharmaceutics 6
materials science 8
decision sciences 4
agricultural and biological sciences 20
physics and astronomy 6
health professions 3
veterinary 4
medicine
nursing 1
dentistry 1
geology 1
immunology and microbiology 1
